In [1]:
%aiida

Loaded AiiDA DB environment.

In [2]:
import nglview
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from IPython.display import display, clear_output, HTML
import nglview

from aiida.orm import Int, Float, Str, Bool

from aiidalab_widgets_base import CodeDropdown, SubmitButtonWidget, StructureBrowserWidget
from aiida.engine import submit, run, run_get_node


#from apps.surfaces.widgets.computer_code_selection import ComputerCodeDropdown
from apps.surfaces.widgets.dft_details_dev import DFTDetails
from apps.surfaces.widgets.viewer_details import ViewerDetails
from apps.surfaces.widgets.slab_validity import slab_is_valid
from apps.surfaces.widgets.suggested_param import suggested_parameters
#from apps.surfaces.widgets.submit_button import SubmitButton

from ase.data import vdw_radii

from apps.surfaces.widgets import analyze_structure
from apps.surfaces.widgets.metadata import MetadataWidget


#ReplicaWorkChain = WorkflowFactory('replica')

from apps.surfaces.reactions.workchains import ReplicaWorkChain

from apps.surfaces.widgets.collective_variable_widget import CollectiveVariableWidget

_ColormakerRegistry()

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
## GENERAL PARAMETERS

the_workchain = ReplicaWorkChain
job_details = {'workchain':'ReplicaWorkChain'}

atoms = None

In [7]:
replica_calc = load_node(458)

In [12]:
list(replica_calc.outputs)

['replica_abc']

In [5]:
builder = ReplicaWorkChain.get_builder()
builder.cp2k_code = load_node(195)
builder.structure = load_node(430)
builder.num_machines = Int(1)
builder.calc_name = Str("h2 test")
#builder.cell
builder.fixed_atoms = Str("")
builder.colvar_targets = Str("1.1 1.2")
builder.target_unit = Str('angstrom')
builder.spring = Float(30.0)
builder.spring_unit = Str('eV/angstrom^2')
builder.subsys_colvar = Dict(dict={'DISTANCE': {'ATOMS': '1 2'}})
builder.calc_type = Str('Full DFT')
builder.mgrid_cutoff = Int(600)
builder.max_force = Float(0.0001)
builder.dftd3_switch = Bool(True)
builder.metadata = {
    "description": "h2 test",
    "label": "ReplicaWorkChain",
}
print("----")
print(builder)
print("----")

----
{'metadata': {'description': 'h2 test', 'label': 'ReplicaWorkChain'}, 'cp2k_code': <Code: Remote code 'cp2k_6.1' on daint-gpu-s904, pk: 195, uuid: 62dc1567-8586-4077-8dd0-29b184597863>, 'structure': <StructureData: uuid: 61f0b08b-e6c2-4c29-8900-21867111cc28 (pk: 430)>, 'num_machines': <Int: uuid: 23b398e5-39fd-4cae-9ac2-2fc8cc7a1bc3 (unstored) value: 1>, 'calc_name': <Str: uuid: b6e99877-69fd-4c8c-bb85-e2daf711e46f (unstored) value: h2 test>, 'fixed_atoms': <Str: uuid: 68632db9-d3b4-4d58-8232-6949284a3522 (unstored) value: >, 'colvar_targets': <Str: uuid: adc3b5b8-9755-4bfd-b3f2-78f34781130d (unstored) value: 1.1 1.2>, 'target_unit': <Str: uuid: 993cf524-f7da-44ae-8682-96c594980f79 (unstored) value: angstrom>, 'spring': <Float: uuid: 379fa3b5-ee0d-45ad-a3f5-ecdc759893c7 (unstored) value: 30.0>, 'spring_unit': <Str: uuid: b42c58f0-7489-4b3d-bd17-729a8960943e (unstored) value: eV/angstrom^2>, 'subsys_colvar': <Dict: uuid: 9100f27b-11e8-43c5-a64b-3c5d6bffbdf4 (unstored)>, 'calc_type'

In [ ]:
node = run(ReplicaWorkChain, **builder)

04/03/2020 02:14:30 PM <8511> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1018|ReplicaWorkChain|initialize]: Init generate replicas
04/03/2020 02:14:30 PM <8511> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1018|ReplicaWorkChain|initialize]: #2 replicas
04/03/2020 02:14:30 PM <8511> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1018|ReplicaWorkChain|next_replica]: Go to replica - 2
04/03/2020 02:14:30 PM <8511> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1018|ReplicaWorkChain|next_replica]: Remaining list: ['1.1', '1.2'] (2)
04/03/2020 02:14:30 PM <8511> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1018|ReplicaWorkChain|generate_replica]: Running CP2K geometry optimization - Target: 1.1
04/03/2020 02:14:31 PM <8511> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1018|ReplicaWorkChain|generate_replica]:  
04/03/2020 02:14:31 PM <8511> aiida.orm.nodes.p

04/03/2020 02:53:46 PM <8511> paramiko.transport: [ERROR] Traceback (most recent call last):
04/03/2020 02:53:46 PM <8511> paramiko.transport: [ERROR]   File "/opt/conda/lib/python3.7/site-packages/paramiko/proxy.py", line 79, in send
04/03/2020 02:53:46 PM <8511> paramiko.transport: [ERROR]     self.process.stdin.write(content)
04/03/2020 02:53:46 PM <8511> paramiko.transport: [ERROR] BrokenPipeError: [Errno 32] Broken pipe
04/03/2020 02:53:46 PM <8511> paramiko.transport: [ERROR] 
04/03/2020 02:53:46 PM <8511> paramiko.transport: [ERROR] During handling of the above exception, another exception occurred:
04/03/2020 02:53:46 PM <8511> paramiko.transport: [ERROR] 
04/03/2020 02:53:46 PM <8511> paramiko.transport: [ERROR] Traceback (most recent call last):
04/03/2020 02:53:46 PM <8511> paramiko.transport: [ERROR]   File "/opt/conda/lib/python3.7/site-packages/paramiko/transport.py", line 2088, in run
04/03/2020 02:53:46 PM <8511> paramiko.transport: [ERROR]     self._channel_handler_tab

04/03/2020 03:06:34 PM <8511> paramiko.transport: [ERROR]     raise ProxyCommandFailure(" ".join(self.cmd), e.strerror)
04/03/2020 03:06:34 PM <8511> paramiko.transport: [ERROR] paramiko.ssh_exception.ProxyCommandFailure: ProxyCommand("ssh -q -Y keimre@ela.cscs.ch netcat daint.cscs.ch 22") returned nonzero exit status: Broken pipe
04/03/2020 03:06:34 PM <8511> paramiko.transport: [ERROR] 
04/03/2020 03:10:05 PM <8511> paramiko.transport: [ERROR] Exception: ProxyCommand("ssh -q -Y keimre@ela.cscs.ch netcat daint.cscs.ch 22") returned nonzero exit status: Broken pipe
04/03/2020 03:10:05 PM <8511> paramiko.transport: [ERROR] Traceback (most recent call last):
04/03/2020 03:10:05 PM <8511> paramiko.transport: [ERROR]   File "/opt/conda/lib/python3.7/site-packages/paramiko/proxy.py", line 79, in send
04/03/2020 03:10:05 PM <8511> paramiko.transport: [ERROR]     self.process.stdin.write(content)
04/03/2020 03:10:05 PM <8511> paramiko.transport: [ERROR] BrokenPipeError: [Errno 32] Broken pipe

04/03/2020 03:19:16 PM <8511> paramiko.transport: [ERROR]   File "/opt/conda/lib/python3.7/site-packages/paramiko/packet.py", line 431, in send_message
04/03/2020 03:19:16 PM <8511> paramiko.transport: [ERROR]     self.write_all(out)
04/03/2020 03:19:16 PM <8511> paramiko.transport: [ERROR]   File "/opt/conda/lib/python3.7/site-packages/paramiko/packet.py", line 336, in write_all
04/03/2020 03:19:16 PM <8511> paramiko.transport: [ERROR]     n = self.__socket.send(out)
04/03/2020 03:19:16 PM <8511> paramiko.transport: [ERROR]   File "/opt/conda/lib/python3.7/site-packages/paramiko/proxy.py", line 85, in send
04/03/2020 03:19:16 PM <8511> paramiko.transport: [ERROR]     raise ProxyCommandFailure(" ".join(self.cmd), e.strerror)
04/03/2020 03:19:16 PM <8511> paramiko.transport: [ERROR] paramiko.ssh_exception.ProxyCommandFailure: ProxyCommand("ssh -q -Y keimre@ela.cscs.ch netcat daint.cscs.ch 22") returned nonzero exit status: Broken pipe
04/03/2020 03:19:16 PM <8511> paramiko.transport: [E

04/03/2020 03:34:04 PM <8511> paramiko.transport: [ERROR]     self.transport._send_user_message(m)
04/03/2020 03:34:04 PM <8511> paramiko.transport: [ERROR]   File "/opt/conda/lib/python3.7/site-packages/paramiko/transport.py", line 1863, in _send_user_message
04/03/2020 03:34:04 PM <8511> paramiko.transport: [ERROR]     self._send_message(data)
04/03/2020 03:34:04 PM <8511> paramiko.transport: [ERROR]   File "/opt/conda/lib/python3.7/site-packages/paramiko/transport.py", line 1839, in _send_message
04/03/2020 03:34:04 PM <8511> paramiko.transport: [ERROR]     self.packetizer.send_message(data)
04/03/2020 03:34:04 PM <8511> paramiko.transport: [ERROR]   File "/opt/conda/lib/python3.7/site-packages/paramiko/packet.py", line 431, in send_message
04/03/2020 03:34:04 PM <8511> paramiko.transport: [ERROR]     self.write_all(out)
04/03/2020 03:34:04 PM <8511> paramiko.transport: [ERROR]   File "/opt/conda/lib/python3.7/site-packages/paramiko/packet.py", line 336, in write_all
04/03/2020 03:3

04/03/2020 04:05:06 PM <8511> paramiko.transport: [ERROR] During handling of the above exception, another exception occurred:
04/03/2020 04:05:06 PM <8511> paramiko.transport: [ERROR] 
04/03/2020 04:05:06 PM <8511> paramiko.transport: [ERROR] Traceback (most recent call last):
04/03/2020 04:05:06 PM <8511> paramiko.transport: [ERROR]   File "/opt/conda/lib/python3.7/site-packages/paramiko/transport.py", line 2088, in run
04/03/2020 04:05:06 PM <8511> paramiko.transport: [ERROR]     self._channel_handler_table[ptype](chan, m)
04/03/2020 04:05:06 PM <8511> paramiko.transport: [ERROR]   File "/opt/conda/lib/python3.7/site-packages/paramiko/channel.py", line 1187, in _handle_close
04/03/2020 04:05:06 PM <8511> paramiko.transport: [ERROR]     self.transport._send_user_message(m)
04/03/2020 04:05:06 PM <8511> paramiko.transport: [ERROR]   File "/opt/conda/lib/python3.7/site-packages/paramiko/transport.py", line 1863, in _send_user_message
04/03/2020 04:05:06 PM <8511> paramiko.transport: [ER

In [5]:
## WIDGETS MONITOR FUNCTIONS
def on_struct_change(c):
    global atoms
    if not struct_browser.results.value:
        return
    job_details['structure']=struct_browser.results.value
    node = struct_browser.results.value
    
    atoms = node.get_ase()
    atoms.pbc = [1, 1, 1]
    
    slab_analyzed = analyze_structure.analyze(atoms)
    job_details['slab_analyzed'] = slab_analyzed
    dft_details_widget.reset()
    
    guess_calc_params(slab_analyzed)
    
    viewer_widget.setup(atoms, slab_analyzed)
    
    with mol_ids_info_out:
        clear_output()
        print(slab_analyzed['summary'])

def on_fixed_atoms_btn_click(c):
    if dft_details_widget.btn_fixed_pressed:
        viewer_widget.show_fixed(dft_details_widget.fixed_atoms.value)
    else:
        viewer_widget.show_fixed("")
        
def guess_calc_params(slab_analyzed):
    method = dft_details_widget.calc_type.value
    valid_slab, msg = slab_is_valid(slab_analyzed,method)
    if valid_slab:        
        atoms_to_fix,num_machines=suggested_parameters(slab_analyzed,method)
        dft_details_widget.reset(fixed_atoms=atoms_to_fix,calc_type=method)
        num_nodes_selector.value = num_machines
    else:
        print(msg)

## Step 1: Select initial structure

In [6]:
## DISPLAY WIDGETS AND DEFINE JOB PARAMETERS

##STRUCTURE
struct_browser = StructureBrowserWidget()
struct_browser.results.observe(on_struct_change, names='value')

##VIEWER
viewer_widget = ViewerDetails()

mol_ids_info_out = ipw.Output()

display(ipw.VBox([struct_browser, viewer_widget, mol_ids_info_out]))

## Step 2: General parameters of the calculation

In [7]:
##CODE
computer_code_dropdown = CodeDropdown(input_plugin='cp2k', path_to_root="../../")

# Misc details
style = {'description_width': '120px'}
layout = {'width': '70%'}
num_nodes_selector = ipw.IntText(
    value=1,
    description='# Nodes',
    style=style, layout=layout)
calc_name_selector = ipw.Text(
    description='Calculation Name: ',
    placeholder='A great name.',
    style=style, layout=layout)

##DFT
dft_details_widget = DFTDetails(job_details=job_details)    
dft_details_widget.btn_fixed_atoms.on_click(on_fixed_atoms_btn_click)
dft_details_widget.calc_type.observe(lambda c: guess_calc_params(), names='value')

# display code dropdown
display(ipw.VBox([computer_code_dropdown, num_nodes_selector, calc_name_selector, dft_details_widget]))

## Step 3: Define collective variable
The unit of the spring constant is [eV/unit of colvar^2].

Three kinds of collective variable are implemented:

**DISTANCE between two atoms**

**ANGLE_PLANE_PLANE between two planes**
- Rotation of plane defined by (P1 - P2) and (P3 - P2) WRT plane defined by a normal .

**BOND_ROTATION**
- Clockwise rotation of (P2-P1) WRT (P4-P3).

In [8]:
cv_widget = CollectiveVariableWidget()

def on_vis_cv_click(c):
    if cv_widget.validation_check() and atoms is not None:
        vis_list = cv_widget.current_cv_instance.visualization_list(atoms)
        viewer_widget.visualize_extra(vis_list)
        
        with mol_ids_info_out:
            cv_widget.current_cv_instance.print_cv(atoms)
        
cv_widget.visualize_colvar_btn.on_click(on_vis_cv_click)

display(cv_widget)

CollectiveVariableWidget(children=(Select(description='Colvar Type', layout=Layout(width='70%'), options={'DIS…

In [11]:
if not struct_browser.results.value:
    print("Please select a structure.")
if not computer_code_dropdown.selected_code:
    print("Please select a code.")

builder = ReplicaWorkChain.get_builder()
builder.cp2k_code = computer_code_dropdown.selected_code
builder.structure = struct_browser.results.value
builder.num_machines = Int(num_nodes_selector.value)
builder.calc_name = Str(calc_name_selector.value)
#builder.cell
builder.fixed_atoms = Str(dft_details_widget.fixed_atoms)
builder.colvar_targets = Str(cv_widget.colvar_targets)
builder.target_unit = Str(cv_widget.target_unit)
builder.spring = Float(cv_widget.spring_constant)
builder.spring_unit = Str(cv_widget.spring_unit)
builder.subsys_colvar = Dict(dict=cv_widget.subsys_colvar)
builder.calc_type = Str(dft_details_widget.calc_type.value)
builder.mgrid_cutoff = Int(dft_details_widget.mgrid_cutoff.value)
builder.max_force = Float(dft_details_widget.max_force.value)
builder.dftd3_switch = Bool(dft_details_widget.vdw_switch.value)

builder.metadata = {
    "description": calc_name_selector.value,
    "label": "ReplicaWorkChain",
}
print("----")
print(builder)
print("----")

----
{'metadata': {'description': 'h2 test', 'label': 'ReplicaWorkChain'}, 'cp2k_code': <Code: Remote code 'cp2k_6.1' on daint-gpu-s904, pk: 195, uuid: 62dc1567-8586-4077-8dd0-29b184597863>, 'structure': <StructureData: uuid: 61f0b08b-e6c2-4c29-8900-21867111cc28 (pk: 430)>, 'num_machines': <Int: uuid: 8856e3db-e11d-4492-9c8a-2e399ecd8bfa (unstored) value: 1>, 'calc_name': <Str: uuid: b7e9d7ae-968e-4318-b35f-0b41e3c807a3 (unstored) value: h2 test>, 'fixed_atoms': <Str: uuid: 49e699a2-fc3a-43cf-b68b-5f007e7d6418 (unstored) value: Text(value='', description='Fixed Atoms', layout=Layout(width='60%'), placeholder='1..10', style=DescriptionStyle(description_width='120px'))>, 'colvar_targets': <Str: uuid: 8a62ffd0-7cd5-4cbc-bd6b-234a9245f064 (unstored) value: 1.0 1.2 1.4 1.6>, 'target_unit': <Str: uuid: dd7afe18-4324-46d7-8b01-5f7bf17b5938 (unstored) value: angstrom>, 'spring': <Float: uuid: 8c27164d-6d86-434a-8724-d2d43fb177d2 (unstored) value: 30.0>, 'spring_unit': <Str: uuid: 06e9f3e8-e64c

In [27]:

def on_submit():
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not computer_code_dropdown.selected_code:
            print("Please select a code.")
            return

        builder = ReplicaWorkChain.get_builder()
        builder.cp2k_code = computer_code_dropdown.selected_code
        builder.structure = struct_browser.results.value
        builder.num_machines = Int(num_nodes_selector.value)
        builder.calc_name = Str(calc_name_selector.value)
        #builder.cell
        builder.fixed_atoms = Str(dft_details_widget.fixed_atoms)
        builder.colvar_targets = Str(cv_widget.colvar_targets)
        builder.target_unit = Str(cv_widget.target_unit)
        builder.spring = Float(cv_widget.spring_constant)
        builder.spring_unit = Str(cv_widget.spring_unit)
        builder.subsys_colvar = Dict(cv_widget.subsys_colvar)
        builder.calc_type = Str(dft_details_widget.calc_type.value)
        builder.mgrid_cutoff = Int(dft_details_widget.mgrid_cutoff.value)
        builder.max_force = Float(dft_details_widget.max_force.value)
        builder.dftd3_switch = Float(dft_details_widget.vdw_switch.value)
        
        builder.metadata = {
            "description": calc_name_selector.value,
            "label": "ReplicaWorkChain",
        }
        print("----")
        print(builder)
        print("----")
        return builder

In [29]:
btn_submit = SubmitButtonWidget(workchain=ReplicaWorkChain,widgets_values=on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)

TypeError: __init__() got an unexpected keyword argument 'workchain'

In [13]:
struct_browser.results.value

<StructureData: uuid: 61f0b08b-e6c2-4c29-8900-21867111cc28 (pk: 430)>

----
{'metadata': {'description': 'h2 test', 'label': 'ReplicaWorkChain'}, 'cp2k_code': <Code: Remote code 'cp2k_6.1' on daint-gpu-s904, pk: 195, uuid: 62dc1567-8586-4077-8dd0-29b184597863>, 'structure': <StructureData: uuid: 61f0b08b-e6c2-4c29-8900-21867111cc28 (pk: 430)>, 'num_machines': <Int: uuid: a2689250-a8c3-43fc-b1ef-c72855701599 (unstored) value: 1>, 'calc_name': <Str: uuid: 103fdbf8-bdd8-4678-ba2f-f51f51124c88 (unstored) value: h2 test>, 'fixed_atoms': <Str: uuid: 01d37643-8a37-4e81-b624-1e5772ac5b5e (unstored) value: >, 'colvar_targets': <Str: uuid: 43c10a5d-9ba1-4cca-aaca-210498579ec3 (unstored) value: 1.1 1.2 1.3>, 'target_unit': <Str: uuid: e9fd7acd-4be9-4861-8534-170cd30546b2 (unstored) value: angstrom>, 'spring': <Float: uuid: 02c5f7a6-d81a-4d73-8486-500ac5189ed7 (unstored) value: 30.0>, 'spring_unit': <Str: uuid: 10bf685e-eedd-47c1-bbef-afd93ad98d1c (unstored) value: eV/angstrom^2>, 'subsys_colvar': <Dict: uuid: ced7f2d5-ba52-40b8-82b0-50e1cc3a71ad (unstored)>, 'calc_t